In [2]:
from bs4 import BeautifulSoup
import requests
import re
from pathlib import Path
import html
import time
import os
import csv
import pandas as pd
import tumblr_crawler as tc
from tumblr_crawler import Snapshot
from fastai.vision.data import verify_images

#Localpath = '/media/larkaa/storage/Pictures/tumblr'
filename = '/media/larkaa/storage/Pictures/tumblr/blog_list.txt'

with open(filename, 'r') as f:
    reader = csv.reader(f)
    sites = [list(line) for line in csv.reader(f)]
sites = sites[1:]
#print(sites[-5:])

In [2]:
sites

[['elmalo82', '', 'deleted'],
 ['thexplicitseu', '', ''],
 ['colorslashform', 'archived', ''],
 ['k-jeezy87', '', 'aka https://www.instagram.com/kjeezy87/?hl=fr'],
 ['chrisbassxx',
  '',
  "'https://www.justusboys.com/forum/threads/424861-Chris-Bass'"],
 ['sweatpantstrackiebulge', '', 'videos not so great'],
 ['weazelas', '', ' #pictures of men'],
 ['scottinno', '', ''],
 ['scottinnooriginals', '', 'artsy'],
 ['jock-boys', '', 'cartoon and boys'],
 ['hung-8inches', '', ''],
 ['justregulardudes', '', 'lots of fatties'],
 ['pennstatefrat', '', '#soft ends at 100'],
 ['alessandrouk', '', '# >5750'],
 ['publicfreaky', '', 'deleted lots of ethnic# ends at 350'],
 ['dudesjackingoff', '', ''],
 ['dickoutinpublic', '', 'good'],
 ['fleshjackit1', '', ''],
 ['bromoerotic', '', ''],
 ['buddybate', '', 'deleted'],
 ['jacknbro', '', 'deleted'],
 ['takeitoffoffoff', '', ''],
 ['beautifulmasculinity', '', 'artsy with labels'],
 ['studinthestreetsslutinthesheets', '', ''],
 ['batcyan', '', 'deleted'],

In [3]:
stop=200000
target = sites[73+13:]
# http://robilwil.tumblr.com/
#http://homemadecock.tumblr.com

target = [['roommate-naked-videos','archived',0]]#,['chrisbassxx','archived',0]]
target = [['homemadecock','',0]]
target = [['simonstalenhag','',0]]
target = [['warblanksexxx','',0]]
target = [['cocksimusmaximus','',0], ['horny-hotguy','',0]]


tot = len(target)
cur = 1
s = tc.get_tor_session()

for site,_,_ in target:
    print('{} of {}'.format(cur,tot),site)
    
    tag_count = tc.scrape(site,
                       update=False,
                       start=0,
                       stop=stop,
                       verbose=True,
                       session=s)
    time.sleep(1)
        
    #tag_count = tc.scrape(site,
    #                   update=False,
    #                   start=0,
    #                   stop=stop,
    #                   verbose=False)
    #time.sleep(1)
    cur+=1
    #location = '/media/larkaa/storage/Pictures/tumblr/'+site
    #verify_images(location,delete=True)
    


	tor ip: 51.89.200.125
1 of 2 cocksimusmaximus
Directory does not exist
800 https://cocksimusmaximus.tumblr.com/api/read?type=photo&num=50&start=800
Ending at  800
2 of 2 horny-hotguy
Directory does not exist
120200 https://horny-hotguy.tumblr.com/api/read?type=photo&num=50&start=120200
Ending at  120200


In [4]:
for site,_,_ in target:
    print('{} of {}'.format(cur,tot),site)
    
    tag_count = tc.scrape(site,
                       update=False,
                       start=0,
                       stop=stop,
                       verbose=True,
                       session=s)
    time.sleep(1)

3 of 2 cocksimusmaximus
685 new images
9 new images
3 of 2 horny-hotguy
1922 new images
1568 new images


In [ ]:
session = requests.session()
r = session.get("https://homemadecock.tumblr.com/api/read?type=photo&num=50&start=13000", timeout=5).text
r

In [ ]:
soup = BeautifulSoup(r, features="lxml")

In [ ]:
test = get_links(soup,'homemadecock')

In [ ]:
a = ['https://farm3.staticflickr.com/2942/15328950065_719d0bf972_o.png', 'https://66.media.tumblr.com/51c5fec7ed9968d93b529bd43a01aa3e/tumblr_nd1870jmlu1s9g3m4o1_500.png']

[x.split('_')[-1].split('.')[0] for x in a if _o.png]

In [ ]:
def get_links(soup,site,update=False):
    img_list = []
    tag_list = []
    caption_list = []
    res = []    
    temp = []
    text = ''
    d = dict()

    for a in soup.find_all('post'):
        photos = [x for x in re.findall(r'(https:[A-Za-z0-9./_]*?(?:\.jpg|\.png|\.gif))', html.unescape(str(a))) 
        if 'avatar' not in x 
        and ('1280' in x or '540' in x or '500' in x) 
        and ' ' not in x]        
        
        if photos:
                
            unique = [x.split('_')[-2] for x in photos]
            unique = list(set(unique))
            #print('unique',unique)
            #print(photos)
            
            for i in unique:
                #find max res photo
                print(i)
                print(photos)
                #print([int(x.split('_')[-1].split('.')[0]) for x in photos if (i in x and '75sq' not in x) and ('o.' not in x)])
                try:
                    max_res = max([int(x.split('_')[-1].split('.')[0]) for x in photos if i in x and '75sq' not in x ])
                except:
                    continue
                photo = [x for x in photos if i in x if str(max_res) in x] [0]
                
                temp.append(photo)
                
                
                # find photos with tags
                for x in a.find_all('tag'):
                    text = re.sub("<[^>]*>", "",x.text)
                    d[photo] = d.get(photo,'') + ';' + text.lower().rstrip('. ')
            
                # find photos with captions
                for x in a.find_all('photo-caption'):
                    
                    text = re.sub("<[^>]*>", "",x.text)
                    d[photo] = d.get(photo,'') + ';' + text.lower().rstrip('. ')

        #print(temp)
        img_list.extend(list(set(temp)))
        
    #convert caption results to dictionary    
    temp =[]
    for key, value in d.items():
        temp = [key,value]
        res.append(temp)
    

    res = pd.DataFrame(res,columns = ['link','tags'])
    tag_list = res
    tag_list['tags'] = res.iloc[:,1].apply(lambda x: re.sub(r"\n", " ",re.sub('http\S*','',x))[1:].rstrip('. '))
    
                
    return(tag_list)

In [ ]:
re.findall('origin":."(.+)"',r)

In [ ]:
#### removing bad videos
vid = '/media/larkaa/storage/Pictures/tumblr/absolutely-scandalous/tumblr_onfopdIaHG1w0dwpv_480.mp4'
#vid = 'HTML, CSS - Lecture 1 - CS50\'s Web Programming with Python and JavaScript-XQs5KcUj-Do.webm'
term = 'MacX'
cmd = "ffmpeg -i {} -f ffmetadata -hide_banner 2>&1 | grep {}".format(vid,term)
cmd = "ffmpeg -i {} -f ffmetadata -hide_banner 2>&1".format(vid)
#cmd = "grep -nr {} rockyou10_new.txt".format(passwd)
print(cmd)
res= ! eval {cmd}


print(any('MacX' in x for x in res))

import os
from os import listdir
from os.path import isfile, join
folder = '/media/larkaa/storage/Pictures/tumblr/'
onlyfiles = []
for root, dirs, files in os.walk(folder): 
        #print(root)
        for filename in files:
            if filename.endswith(('.mp4','mkv','avi')):
                onlyfiles.append(os.path.join(root,filename))
                    
#onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f)) if f.endswith('mp4')]
print(onlyfiles)

In [ ]:
base_url = "https://{0}.tumblr.com/api/read?type={1}&num={2}&start={3}"

test = base_url.format('cbxx12', 'photo', 50, 0)
r = requests.get(test)
print(test)
print(r.text)

In [ ]:
#print(r.text)
# checking with xml....
import html
import xmltodict

from bs4 import BeautifulSoup

rh = html.unescape(r.text)
response = urllib.request.urlopen(media_url)
soup = BeautifulSoup(response.read(), from_encoding=response.headers.get_content_charset())

temp = soup.find_all('post')
    
xml_cleaned = print(re.sub(u'[^\x20-\x7f]+', u'', r.content.decode('utf-8')))

data = xmltodict.parse(xml_cleaned)
posts = data["tumblr"]["posts"]["post"]
for post in posts:
    print(post)
            

In [ ]:

#trying new site

#http://www.queerclick.com/qc/2019/09/gwips-top-ten-of-the-week-110.php

import requests 
from bs4 import BeautifulSoup
site = 'http://guyswithiphones.com/'
#r = requests.get(site)
#r.text
soup = BeautifulSoup(r.text)

#<li><a href="http://guyswithiphones.com/2019/09/437804/" 
#name="437804" class="dlnk">

mydivs = soup.findAll("a", {"class": "dlnk"})

#for a in mydivs:
#    print(a.get('href'))
    

# link to page, from then saved under 'realimg' / 'bigimg'
#<div id="bigimg">
#<span class="realimage"></span><img src="http://guyswithiphones.com/upload/user-2019/e84c9173-362f-491f-adee-f590d785be57.png" width="1125" height="2436" 
#alt="Guys with iPhones" />
#</div>

site2 = mydivs[0].get('href')
print(site2)
#r2 = requests.get(site2)
soup2 = BeautifulSoup(r2.text)


mydivs2 = soup2.findAll("img", {"alt": "Guys with iPhones"})
print(mydivs2[0].get('src'))

#<div class="moreScroller" style="width: 85px;">
#<a href="http://guyswithiphones.com/2019/09/437658/"><img src="http://guyswithiphones.com/assets_c/2019/09/35459502_1845486752184383-thumb-76x76-302662.jpg" width="75"></a>
#</div>

mydivs3 = soup2.findAll("div", {"class": "moreScroller"})
for child in mydivs3:
    print(child.find('a').get('href'))

#for child in raw_quality:
#    grade = child.find(class_ = 'grade').get_text()





#get all other pages
#span class="yearboxes">
websites = [site]
print()
mydivs4 = soup.findAll("span", {"class": "yearboxes"})
for child in mydivs4:
    #print(child)
    temp = (child.findAll('a'))
    for i in temp:
        websites.append(i.get('href'))



In [4]:
# crawl guyswithiphones
import pandas as pd
def crawl_gwip():
    site = 'http://guyswithiphones.com/'
    websites = [site]
    soup = BeautifulSoup(requests.get(site).text)

    ws = soup.findAll("span", {"class": "yearboxes"})
    for child in ws:
        temp = (child.findAll('a'))
        for i in temp:
            websites.append(i.get('href'))
    photos = []
    for w in websites:
        print(w,end=' ')
        photos.extend(gwip_all_pages(w))
    print(len(photos))
    df = pd.DataFrame(photos)
    df.to_csv('gwip_links.txt', index=False)

#count = 1
def gwip_all_pages(site):
    global count
    
    photos = []
    
    #1 get website 
    r = requests.get(site)
    soup = BeautifulSoup(r.text)
    
    
    # 2 get first level links
    
    links_1 = soup.findAll("a", {"class": "dlnk"})
    
    #3 for each link, loop through to get photos
    for l in links_1:
        site2 = l.get('href')
        #if count%1000==0:print(count,len(photos))
        
        while True:
            r2 = requests.get(site2)
            soup2 = BeautifulSoup(r2.text)
            
            mydivs2 = soup2.findAll("img", {"alt": lambda L: L and "Guys with iPhones" in L})
            photos.append(mydivs2[0].get('src'))
            
            # get other links if exist
            mydivs3 = soup2.findAll("div", {"class": "moreScroller"})
            if mydivs3 and mydivs3[0].find('a'):
                site2 = mydivs3[0].find('a').get('href')
                #count+=1
            else:
                break
            break
                
    print(len(photos))
    return photos
        
    
    

In [8]:
df = pd.DataFrame(photos)
df.to_csv('gwip_links.txt', index=False)

In [10]:
df.head()


,0
0,https://guyswithiphones.com/upload/user-2020/i...
1,https://guyswithiphones.com/2020/04/05/oxh59nc...
2,https://guyswithiphones.com/2020/04/05/oxyc6mS...
3,https://guyswithiphones.com/2020/04/05/oolbncF...
4,https://guyswithiphones.com/2020/04/05/ooohr0S...


In [7]:
site = 'http://guyswithiphones.com/'
websites = [site]
soup = BeautifulSoup(requests.get(site).text)

ws = soup.findAll("span", {"class": "yearboxes"})
for child in ws:
    #print(child)
    temp = (child.findAll('a'))
    for i in temp:
        websites.append(i.get('href'))

photos = []
#websites= ['http://guyswithiphones.com/2018/02/28/']
for w in websites:
    print(w,end=' ')
    photos.extend(crawl_gwip(w))
    #print(len(photos))

print(len(photos))

http://guyswithiphones.com/ 48
https://guyswithiphones.com/2020/01/31/ 16
https://guyswithiphones.com/2020/02/29/ 15
https://guyswithiphones.com/2020/03/31/ 16
https://guyswithiphones.com/2020/04/05/ 9
https://guyswithiphones.com/2019/01/31/ 16
https://guyswithiphones.com/2019/02/28/ 16
https://guyswithiphones.com/2019/03/31/ 17
https://guyswithiphones.com/2019/04/30/ 16
https://guyswithiphones.com/2019/05/31/ 16
https://guyswithiphones.com/2019/06/30/ 16
https://guyswithiphones.com/2019/07/31/ 16
https://guyswithiphones.com/2019/08/31/ 18
https://guyswithiphones.com/2019/09/30/ 17
https://guyswithiphones.com/2019/10/31/ 18
https://guyswithiphones.com/2019/11/30/ 17
https://guyswithiphones.com/2019/12/31/ 16
https://guyswithiphones.com/2018/01/31/ 51
https://guyswithiphones.com/2018/02/28/ 42
https://guyswithiphones.com/2018/03/31/ 58
https://guyswithiphones.com/2018/04/30/ 60
https://guyswithiphones.com/2018/05/31/ 61
https://guyswithiphones.com/2018/06/30/ 22
https://guyswithiphones.

SSLError: HTTPSConnectionPool(host='guyswithiphones.com', port=443): Max retries exceeded with url: /2012/08/243474/ (Caused by SSLError(SSLError("bad handshake: SysCallError(110, 'ETIMEDOUT')")))

In [ ]:
import urllib
import urllib.request
from pathlib import Path
import os

def dl_img_list(lst,path):
    RETRY = 3
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

    
    
    if not os.path.exists('{}'.format(path)):
        os.makedirs("{}".format(path))
        print('Creating new directory')
        
    print(len(lst))
    #count=1
    for item in lst:
        
        dl_name = os.path.join("{}".format(path),item.split('/')[-1]) 
        
        my_file = Path(str(dl_name))
        #if my_file.is_file(): print(my_file)
        #print(my_file)
        if not my_file.is_file():
            retry_times = 0
            while retry_times < RETRY:
                try:
                    with urllib.request.urlopen(item) as response, open(dl_name, 'wb') as out_file:
                        data = response.read() # a `bytes` object
                        out_file.write(data)

                    break;
                except:
                    retry_times+=1 
                    pass
                retry_times+=1
            #count+=1
    #print(count)
                
                

In [ ]:
## pornhub serach and save videos
def search_pornhub(search_term):
    s_term = search_term.split(' ').join('+')
    url = 'https://www.pornhub.com/gay/video/search?search='+s_term
    r = requests.get(url)
    res = re.findall('"(/view_video\.php.*?)".*?title="(.*?)"',r.text)
    d={}
    for value,key in res:
        d[key] = d.get(key, value)
        
    count=1
    for key, val in d.items():
        i = 'https://www.pornhub.com'+val
        cmd = 'youtube-dl {}'.format(i)
        print(count, key)
        res = ! eval {cmd}
        count+=1
    print('Downloaded {} videos'.format(count))

In [ ]:

    #print(i)

In [ ]:
print(sites)

In [ ]:
usernames = ["Awalpe75", "bodyheat75", "scruffy03", "thirty20", "elfton94", "venustus", "jmecparis", "windy", "dlishdd", "nelson7", "Matmat", "hung69olly", "Exotic_30", "arco", "bernexx", "kida26", "Tonio", "dudehayd", "botom59", "nickine", "~elektra~", "lebstud22", "Kunene", "Builtguy", "alala", "crbllm", "mikael83", "Arcon", "n2kinkdude", "mek_a_paris", "hotness84", "Gingernick", "ASB48", "gin+tonic", "grego75", "TheGoodBoy", "calirunner", "jock_model", "NYCVersGuy", "SwErik", "Frenchkik", "galfi14", "Rebel Yell", "pyramides", "paul660252", "Pronoia", "BeDe", "terrick", "big_blue", "Blitz-Ace", "madijames", "fldbgfun", "Bahquarius", "Dude1984", "rustola", "Ricardo29", "Exposure", "coolkid22", "blue_impul", "koni", "crazyman83", "Stargazer1", "Clivus", "fr_nudist", "candente1", "JNJ", "arc30", "Mi-Dou", "pussyboy4u", "bon_matos", "minac", "Sentry5", "officexxx", "Abnihilo", "uknik", "ACurtis802/Breakthru", "cuki", "Ollie09", "Reflet", "crewkid", "nickine", ]
res = []

for i in usernames:
    cmd = 'grep -nr {} /media/larkaa/storage/Torrents/BreachCompilation/data/'.format(i)
    print(cmd)
    temp= ! eval {cmd}
    print(temp)
    res.append(temp)

In [ ]:
def get_vid_links(soup,site):
    vid_list = []
    d = dict()
    a=''

    #search all posts
    for b in soup.find_all('post'):
        #find vid links
        temp = re.findall('source src=\"(https:.*?)\"'  , str(b))
        if temp:
            temp = temp[0]
            if '480' not in temp:
                #a_temp='https://vt.tumblr.com/'+(temp.split('/')[-1])
                a_temp='https://ve.media.tumblr.com/'+(temp.split('/')[-1])
            else:
                #a_temp='https://vt.tumblr.com/'+(temp.split('/')[-2]+"_480")
                a_temp='https://ve.media.tumblr.com/'+(temp.split('/')[-2]+"_480")
            if a_temp.endswith('.mp4'):
                a = a_temp
            else:
                a = a_temp + '.mp4'
            
            #add photo first, then include tags
            d[a] = d.get(a,'')       
            # find photos with tags
            for x in b.find_all('tag'):
                text = re.sub("<[^>]*>", "",x.text)
                d[a] = d.get(a,'') + ';' + text.lower().rstrip('. ')
             
            # find photos with captions
            for x in b.find_all('photo-caption'):
                text = re.sub("<[^>]*>", "",x.text)
                d[a] = d.get(a,'') + ';' + text.lower().rstrip('. ')
                        
    res =[]
    for key, value in d.items():
        temp = [key,value[1:]]
        res.append(temp)

    return(res)

In [ ]:
file = '/media/larkaa/storage/Pictures/tumblr/roommate-naked-videos/ve.media.tumblr.com_480.mp4'

with open(file, 'rb') as f:
    first = f.read(243)
    
print(first)
print(re.findall('Denied',str(first)))